<a href="https://colab.research.google.com/github/talpt/pyton/blob/main/Maestro_Formasyon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/rongardF/tvdatafeed  matplotlib openpyxl
!pip install tradingview-screener==2.5.0
!pip install tvDatafeed ta

import pandas as pd
import numpy as np
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
import ta
import matplotlib.pyplot as plt
import os
import warnings

warnings.filterwarnings("ignore")

# TradingView'e giriş yapmadan bağlanın
tv = TvDatafeed()

# Türkiye piyasasındaki tüm sembolleri almak için
try:
    # Tüm sembolleri TradingView'den çekme
    Hisseler = get_all_symbols(market='turkey')
    Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]
    Hisseler = sorted(Hisseler)
    print(f"{len(Hisseler)} adet hisse senedi bulundu.")
except Exception as e:
    print(f"Hata alındı: {e}")

# Elmas formasyonu kontrol fonksiyonu
def elmas_formasyonu(df):
    high = df['high']
    low = df['low']

    genişleme = (high.diff().mean() > 0) and (low.diff().mean() < 0)
    daralma = (high.diff().mean() < 0) and (low.diff().mean() > 0)

    return genişleme and daralma

# OBO formasyonu kontrol fonksiyonu
def obo_formasyonu(df):
    high = df['high']
    if len(high) < 5:
        return False
    sol_omuz = high.iloc[-5]
    baş = high.iloc[-3]
    sağ_omuz = high.iloc[-1]

    return baş > sol_omuz and baş > sağ_omuz and abs(sol_omuz - sağ_omuz) / baş < 0.05

# TOBO formasyonu kontrol fonksiyonu
def tobo_formasyonu(df):
    low = df['low']
    if len(low) < 5:
        return False
    sol_omuz = low.iloc[-5]
    baş = low.iloc[-3]
    sağ_omuz = low.iloc[-1]

    return baş < sol_omuz and baş < sağ_omuz and abs(sol_omuz - sağ_omuz) / baş < 0.05

# Flama formasyonu kontrol fonksiyonu
def flama_formasyonu(df):
    high = df['high']
    low = df['low']

    daralan_kanal = (high.diff().mean() < 0) and (low.diff().mean() > 0)
    son_kırılma = high.iloc[-1] > high.mean() or low.iloc[-1] < low.mean()

    return daralan_kanal and son_kırılma

# Üçgen (Triangles) kontrol fonksiyonu
def ucgen_formasyonu(df):
    high = df['high']
    low = df['low']

    # Fiyat sıkışması: high ve low arasında daralma
    daralan_kanal = (high.diff().mean() < 0) and (low.diff().mean() < 0)

    # Kırılma
    son_kırılma = high.iloc[-1] > high.mean() or low.iloc[-1] < low.mean()

    return daralan_kanal and son_kırılma

# Kupa ve Sap (Cup and Handle) kontrol fonksiyonu
def kupa_sap_formasyonu(df):
    low = df['low']
    if len(low) < 10:
        return False
    dip = low.min()
    son_dip = low.iloc[-1]

    return son_dip > dip  # Son dip önceki dipten yüksek mi?

# Yükselen Kanal (Rising Channel) kontrol fonksiyonu
def yukselen_kanal_formasyonu(df):
    high = df['high']
    low = df['low']

    return high.diff().mean() > 0 and low.diff().mean() > 0

# Düşen Kanal (Falling Channel) kontrol fonksiyonu
def dusen_kanal_formasyonu(df):
    high = df['high']
    low = df['low']

    return high.diff().mean() < 0 and low.diff().mean() < 0

# Dikdörtgen (Rectangle) kontrol fonksiyonu
def dikdortgen_formasyonu(df):
    high = df['high']
    low = df['low']

    return high.max() - low.min() < (high.mean() * 0.05)  # Fiyat aralığı daralmış mı?

# Formasyon tarama fonksiyonu
def tarama_algoritmalari(df, formasyon):
    if formasyon == 'elmas':
        return elmas_formasyonu(df)
    elif formasyon == 'flama':
        return flama_formasyonu(df)
    elif formasyon == 'obo':
        return obo_formasyonu(df)
    elif formasyon == 'tobo':
        return tobo_formasyonu(df)
    elif formasyon == 'ucgen':
        return ucgen_formasyonu(df)
    elif formasyon == 'kupa_sap':
        return kupa_sap_formasyonu(df)
    elif formasyon == 'yukselen_kanal':
        return yukselen_kanal_formasyonu(df)
    elif formasyon == 'dusen_kanal':
        return dusen_kanal_formasyonu(df)
    elif formasyon == 'dikdortgen':
        return dikdortgen_formasyonu(df)
    return False

# Hisse senetleri üzerinde tarama yapan fonksiyon
def tarama_yap(hisseler, formasyon, interval):
    for hisse in hisseler:
        try:
            # TradingView'den hisse verilerini al
            df = tv.get_hist(symbol=f'BIST:{hisse}', interval=interval, n_bars=660)

            # Formasyonu kontrol et
            if tarama_algoritmalari(df, formasyon):
                print(f"{hisse} için {formasyon} formasyonu bulundu.")
        except Exception as e:
            print(f"{hisse} için veri çekme hatası: {e}")

# Kullanıcının formasyon seçimi
def formasyon_secimi():
    formasyonlar = {
        '1': 'elmas',
        '2': 'flama',
        '3': 'obo',
        '4': 'tobo',
        '5': 'ucgen',
        '6': 'kupa_sap',
        '7': 'yukselen_kanal',
        '8': 'dusen_kanal',
        '9': 'dikdortgen'
    }

    print("Lütfen taramak istediğiniz formasyonu seçin:")
    for key, formasyon in formasyonlar.items():
        print(f"{key}: {formasyon.capitalize()} Formasyonu")

    secim = input("Seçiminizi yapın (1-9): ")
    return formasyonlar.get(secim, 'elmas')

# Kullanıcının interval seçimi
def interval_secimi():
    interval_map = {
        '1': Interval.in_15_minute,
        '2': Interval.in_30_minute,
        '3': Interval.in_45_minute,
        '4': Interval.in_1_hour,
        '5': Interval.in_2_hour,
        '6': Interval.in_4_hour,
        '7': Interval.in_daily,
        '8': Interval.in_weekly,
        '9': Interval.in_monthly
    }

    print("\nLütfen zaman dilimini (interval) seçin:")
    print("1: 15 Dakika")
    print("2: 30 Dakika")
    print("3: 45 Dakika")
    print("4: 1 Saat")
    print("5: 2 Saat")
    print("6: 4 Saat")
    print("7: Günlük")
    print("8: Haftalık")
    print("9: Aylık")
    secim = input("Seçiminizi yapın (1-9): ")
    return interval_map.get(secim, Interval.in_daily)  # Varsayılan olarak günlük

# Main function
def main():
    # Kullanıcının formasyon seçimi
    formasyon = formasyon_secimi()

    # Kullanıcının interval seçimi
    interval = interval_secimi()

    # Tüm BIST hisse senetleri üzerinde formasyon taraması
    tarama_yap(Hisseler, formasyon, interval)


# Programı başlat
main()

  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-tlkocwpf
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-tlkocwpf
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done
  Created wheel for tvdatafeed: filename=tvdatafeed-2.1.0-py3-none-any.whl size=17533 sha256=1a1af227d30ded228ccf433360cd79df96c7c3532df52a5b05d30b056cdcf75f
  Stored in directory: /tmp/pip-ephem-wheel-cache-zgulldcl/wheels/e4/32/1e/21ebcacc6549d75fae3bf3ff75cee8fcbe4e5c189d88fbd5a1
Successfully built tvdatafeed
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=2aa25cf3609e6012b6b309842c3bfa6b3c2a756debb55a0e4aa234fd88be4342
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a

583 adet hisse senedi bulundu.
Lütfen taramak istediğiniz formasyonu seçin:
1: Elmas Formasyonu
2: Flama Formasyonu
3: Obo Formasyonu
4: Tobo Formasyonu
5: Ucgen Formasyonu
6: Kupa_sap Formasyonu
7: Yukselen_kanal Formasyonu
8: Dusen_kanal Formasyonu
9: Dikdortgen Formasyonu
Seçiminizi yapın (1-9): 2

Lütfen zaman dilimini (interval) seçin:
1: 15 Dakika
2: 30 Dakika
3: 45 Dakika
4: 1 Saat
5: 2 Saat
6: 4 Saat
7: Günlük
8: Haftalık
9: Aylık
Seçiminizi yapın (1-9): 4
AGHOL için flama formasyonu bulundu.


KeyboardInterrupt: 